In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab


In [2]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import get_file
import numpy as np
import random
import sys
import io
import requests
import re
import pandas as pd

In [3]:
df_preguntas=pd.read_csv("preguntas_cleaned.csv")
df_preguntas["QnA"]=df_preguntas[["Pregunta","Respuesta"]].agg(u'\n'.join,axis=1)
raw_text="\n".join(df_preguntas["QnA"])
promedio=df_preguntas["QnA"].str.len().mean()
print(raw_text[0:1000])

 muy buenos días, presidente y funcionarios que lo acompañan.ramón flores, de el centinela informa y the human hoy.presidente, usted y el ministro presidente de la suprema corte de justicia dieron instrucciones de revisar el caso de claudia ivonne sánchez mayorga. le informo que con toda saña la encargada de despacho del juzgado sexto penal, lorena marisol aranda millán, condenó a claudia a 22 años de cárcel simplemente con las imputaciones verbales de las mujeres que trabajaban libremente en el restaurante, pero extrañamente no tiene declaración patrimonial en el portal del consejo de la judicatura y los agentes del ministerio público tampoco.presidente, este caso lo había usted pedido que lo revisara alejandro encinas, que lo vieran otras personas.pero aquí viene que precisamente yo había aquí acusado a terea ulloa de que estaba reclamándole dinero al estado y hoy, en contubernio con lo que es el poder judicial de la federación, la fiscalía general de la república, condenan a claudia

In [4]:
processed_text = raw_text.lower()

print('corpus length:', len(processed_text))

chars = sorted(list(set(processed_text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 765198
total chars: 64


In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 100
step = 3
sentences = []
next_chars = []
for i in range(0, len(processed_text) - maxlen, step):
    sentences.append(processed_text[i: i + maxlen])
    next_chars.append(processed_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 255033


In [6]:
sentences

['\xa0muy buenos días, presidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y',
 'y buenos días, presidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0th',
 'uenos días, presidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0the\xa0h',
 'os días, presidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0the\xa0huma',
 'días, presidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0the\xa0human h',
 's, presidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0the\xa0human hoy.',
 'presidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0the\xa0human hoy.pre',
 'sidente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0the\xa0human hoy.presid',
 'ente y funcionarios que lo acompañan.ramón flores, de\xa0el centinela informa y\xa0the\xa0hu

In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
Total params: 107,072
Trainable params: 107,072
Non-trainable params: 0
_________________________________________________________________


In [10]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("******************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(processed_text) - maxlen - 1)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        generated = ''
        sentence = processed_text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [12]:
# Ignore useless W0819 warnings generated by TensorFlow 2.0.  Hopefully can remove this ignore in the future.
# See https://github.com/tensorflow/tensorflow/issues/31308
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Fit the model
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
1993/1993 [==============================] - ETA: 0s - loss: 1.7902******************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: "o la guerra de ucrania, para decirlo de la mejor manera, porque nosotros en eso somos muy respetuoso"
o la guerra de ucrania, para decirlo de la mejor manera, porque nosotros en eso somos muy respetuosos de la resisto de la despete a los desde el despues de la porte a lo que se hay a lo metio es el probes que se hay que se hay a los de desde la para consertar a los de sus por los de consertar a la para de la problema de la portuna de la despeto de la propueta de la constitución de la portuna de la constitución de la despete a la presidente, es una de la despues de la desperta de la gente de la p
----- temperature: 0.5
----- Generating with seed: "o la guerra de ucrania, para decirlo de la mejor manera, porque nosotros en eso somos muy respetuoso"
o la guerra de ucrani

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


orque está la construcción, porque está a la construcción, porque no se llega a la presenta de la construcción, está la construcción, en la respuesta de la construcción, el presidente de la construcción, el presidente en la corrupción de la construcción.    
 sí, estamos hablando de la construcción.    
 sí, sí, es que está el presidente de la construcción de la construcción, 
----- temperature: 0.5
----- Generating with seed: " vinculado con su asesinato.este es un fotógrafo de tijuana, que es víctima de una banda; ya hay 10 "
 vinculado con su asesinato.este es un fotógrafo de tijuana, que es víctima de una banda; ya hay 10 de los delitos, pero si no va a entregar, están porque los trabajadores no hay presidente de que se está ayuda a la eda construcción.    
 sí, los estados de los presoles porque es todos los partismos con el compromiso de descriencia de las convenientes, porque está a alguna empresa de hacienda de los presidentes de la planta de corrupción.    
 ah, no, es alguno 